# subprocess
- The subprocess module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.  [Documentation: subprocess][1]
[1]: <https://docs.python.org/3/library/subprocess.html>

In [1]:
from subprocess import run, Popen, PIPE

## 1. Run system command

In [2]:
# Run command `ls -l`
r1 = run(['ls', '-l'], stdout=PIPE)  # Need to separate command & argument (flag)
r2 = run('ls -l', shell=True)  # Shell mode, not recommanded due to security concern like "shell injection"
print(r1.stdout)
print("Output for r1 is captured, r2 is not.")

b'total 36\n-rw-rw-r-- 1 qz qz 5383 Jun 17 21:55 decorator.ipynb\n-rw-rw-r-- 1 qz qz 1070 Jun 15 03:44 LICENSE\n-rw-rw-r-- 1 qz qz 4011 Jun 17 23:49 lru_cache.ipynb\n-rw-rw-r-- 1 qz qz  140 Jun 17 23:49 README.md\n-rw-rw-r-- 1 qz qz 7591 Jun 15 03:45 sched.ipynb\n-rw-rw-r-- 1 qz qz 5661 Jun 18 23:29 subprocess.ipynb\n'
Output for r1 is captured, r2 is not.


### 1.0. shell injection example

In [3]:
# https://docs.python.org/3/library/subprocess.html#security-considerations
filename = 'somefile; ls -l'
command = 'ls -l {}'.format(filename)
print('No quote: {}'.format(command))
from shlex import quote  # Avoid shell injection
command = 'ls -l {}'.format(quote(filename))
print('Use quote: {}\n'.format(command))


# need to quote(command) before split
remote_command = 'ssh home {}'.format(quote(command))
print('Quoted command: {}'.format(remote_command))
from shlex import split
remote_command = split(remote_command)
print('First split: {}'.format(remote_command))
command = split(remote_command[-1])
print('Second split: {}'.format(command))

No quote: ls -l somefile; ls -l
Use quote: ls -l 'somefile; ls -l'

Quoted command: ssh home 'ls -l '"'"'somefile; ls -l'"'"''
First split: ['ssh', 'home', "ls -l 'somefile; ls -l'"]
Second split: ['ls', '-l', 'somefile; ls -l']


## 2. Popen & communicate

In [4]:
# Popen: Execute child program in a new process
# PIPE: Special value that can be used as the stdin, 
#   stdout or stderr argument to Popen and indicates 
#   that a pipe to the standard stream should be opened.
# communicate(): returns tuple (stdout_data, stderr_data)
p = Popen(['echo', "Hello World"], stdout=PIPE)
out, err = p.communicate()  
print("Standard output: {}".format(out))
print("Standard error: {}".format(err))

Standard output: b'Hello World\n'
Standard error: None


### 2.0. Standard Error

In [5]:
import subprocess
try:
    subprocess.run(['false'], check=True)
except subprocess.CalledProcessError as err:
    print('ERROR:', err)

ERROR: Command '['false']' returned non-zero exit status 1.


## 3. Pipe `|`

In [6]:
p1 = Popen(["ls", "-l"], stdout=PIPE)
p2 = Popen(["grep", "subprocess.*"], stdin=p1.stdout, stdout=PIPE)  # stdout to null, set stdout to: subprocess.DEVNULL
p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
out, err = p2.communicate()
print("Standard output: {}".format(out))
print("Standard error: {}".format(err))

Standard output: b'-rw-rw-r-- 1 qz qz 5661 Jun 18 23:29 subprocess.ipynb\n'
Standard error: None
